In [2]:
def create_circle(m,r):
    return patches.Circle((m,0), radius=r, angle=180.,lw=2,fill=False)

In [2]:
def principal_to_pq(s_1fe,s_3fe):
    return (s_1fe + s_3fe)/2, (s_1fe - s_3fe)/2

In [2]:
def find_b_tanalpha(s_1fe,s_3fe):
    s_m, t_m = principal_to_pq(s_1fe,s_3fe)
    regr = sp.stats.linregress(s_m,t_m)
    return regr.slope, regr.intercept

In [4]:
def find_c_sinphi(s_1fe,s_3fe):
    s_m, t_m = principal_to_pq(s_1fe,s_3fe)
    regr = sp.stats.linregress(s_m,t_m)
    return regr.slope, regr.intercept/np.cos(np.arcsin(regr.slope))

In [4]:
def find_failure_stresses(s_1fe,s_3fe):
    sinphi, c = find_c_sinphi(s_1fe,s_3fe)
    phi = np.arcsin(sinphi)
    theta = np.pi/4 + phi/2
    tau_alpha = (c + s_3fe*np.tan(phi))/(1-np.tan(phi)/np.tan(theta))
    s_alpha = (tau_alpha - c)/np.tan(phi)
    return s_alpha, tau_alpha

In [2]:
def plot_basic_circles(s_1fe,s_3fe):
    fig, ax = plt.subplots(figsize=(12,12))
    s_m, t_m = principal_to_pq(s_1fe,s_3fe)
    for i in range(len(s_1fe)):
        ax.add_patch(create_circle(s_m[i],t_m[i]))

    ax.set(aspect='equal')
    ax.set_xlabel("$\\sigma'_\\mathrm{n}$ / kPa")
    ax.set_ylabel('$\\tau$ / kPa')
    #ax.legend(fontsize=12)
    ax.set_xlim(0,np.max(s_1fe))
    ax.set_ylim(0,np.max(s_1fe)/2)
    #ax.spines['left'].set_position(('data', 0))
    fig.tight_layout()
    plt.show();

In [ ]:
def plot_pq(s_1fe,s_3fe,u=0):
    s_m, t_m = principal_to_pq(s_1fe,s_3fe)
    m, b = find_b_tanalpha(s_1fe,s_3fe)
    
    fig, ax = plt.subplots(figsize=(12,8))
    ax.set_xlabel("$\\sigma_\\mathrm{M}$ / kPa")
    ax.set_ylabel("$(\\sigma_1 - \\sigma_3) / 2$ / kPa")
    ax.plot(s_m,t_m,ls='',marker='d',label="$\\sigma_\\mathrm{M} = (\\sigma_1'+\\sigma_3')/2")
    
    ax.plot([0,np.max(s_m)],[b,b+m*np.max(s_m)],label="$b' + \\sigma_\\mathrm{M}' \\tan \\alpha'$")
    fig.tight_layout()
    ax.set_xlim(0,np.max(s_m)*1.02)
    ax.set_ylim(0,np.max(t_m)*1.02)
    plt.show();

In [3]:
def plot_circles_tangent(s_1fe,s_3fe):
    fig, ax = plt.subplots(figsize=(12,12))
    s_m, t_m = principal_to_pq(s_1fe,s_3fe)
    sinphi, c = find_c_sinphi(s_1fe,s_3fe)
    for i in range(len(s_1fe)):
        ax.add_patch(create_circle(s_m[i],t_m[i]))

    ax.plot([0,np.max(s_1fe)],[c,c+np.max(s_1fe)*np.tan(np.arcsin(sinphi))],label="$c' + \\sigma_\\mathrm{n}' \\tan \\varphi'$")
    ax.set(aspect='equal')
    ax.set_xlabel("$\\sigma'_\\mathrm{n}$ / kPa")
    ax.set_ylabel('$\\tau$ / kPa')
    ax.legend()
    ax.set_xlim(0,np.max(s_1fe))
    ax.set_ylim(0,np.max(s_1fe)/2)
    #ax.spines['left'].set_position(('data', 0))
    fig.tight_layout()
    plt.show();

In [ ]:
def plot_circles_failure(s_1fe,s_3fe):
    fig, ax = plt.subplots(figsize=(12,12))
    s_m, t_m = principal_to_pq(s_1fe,s_3fe)
    sinphi, c = find_c_sinphi(s_1fe,s_3fe)
    s_a, t_a = find_failure_stresses(s_1fe,s_3fe)
    for i in range(len(s_1fe)):
        ax.add_patch(create_circle(s_m[i],t_m[i]))

    ax.plot([0,np.max(s_1fe)],[c,c+np.max(s_1fe)*np.tan(np.arcsin(sinphi))],label="$c' + \\sigma_\\mathrm{n}' \\tan \\varphi'$")
    ax.plot(s_a,t_a,ls='',marker='o', label="$\\sigma_\\vartheta', \\tau_\\vartheta$",color='red')
    for i in range(len(s_1fe)):
        ax.plot([s_3fe[i],s_a[i]],[0,t_a[i]],ls='--',color='red')
    ax.set(aspect='equal')
    ax.set_xlabel("$\\sigma'_\\mathrm{n}$ / kPa")
    ax.set_ylabel('$\\tau$ / kPa')
    ax.legend()
    ax.set_xlim(0,np.max(s_1fe))
    ax.set_ylim(0,np.max(s_1fe)/2)
    #ax.spines['left'].set_position(('data', 0))
    fig.tight_layout()
    plt.show();